### Import and install

In [352]:
import nltk
import string
import pandas as pd
import numpy as np
import copy
import re

from tqdm import tqdm
from collections import defaultdict, Counter
nltk.download('words')

[nltk_data] Downloading package words to /Users/jackcook/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### Get 5 letter words

Downloaded from https://www.powerlanguage.co.uk/wordle/main.e65ce0a5.js, the former (shorter) array of ~2,500 is [apparently the list of possible answers](https://puzzling.stackexchange.com/questions/114419/what-dictionary-is-wordle-based-on), whereas there are ~10,000 other valid guesses.

For our use case, we only care about answers as potential words.

In [131]:
with open('answers.txt') as file:
    possible_answers = file.readlines()

list_possible_answers = sorted([re.sub(r'[^A-Z]', '', t.upper()) for t in possible_answers[0].split(',')])
print(len(list_possible_answers),
      list_possible_answers[:5])

2315 ['ABACK', 'ABASE', 'ABATE', 'ABBEY', 'ABBOT']


Arrange as a DataFrame

In [136]:
arr_words_5l = np.array([list(w) for w in list_possible_answers])
df_words_5l = pd.DataFrame(data=arr_words_5l,
                           columns=[f'letter_{i+1}' for i in range(5)])
df_words_5l['word'] = list_possible_answers
df_words_5l.head()

,letter_1,letter_2,letter_3,letter_4,letter_5,word
0,A,B,A,C,K,ABACK
1,A,B,A,S,E,ABASE
2,A,B,A,T,E,ABATE
3,A,B,B,E,Y,ABBEY
4,A,B,B,O,T,ABBOT


Assess how frequent letters are at each position

In [137]:
test_dict_letter_counts = Counter()
for i in range(5):
    test_dict_letter_counts[i+1] = Counter(df_words_5l[f'letter_{i+1}'])
    
test_dict_letter_counts[1]

Counter({'A': 141,
         'B': 173,
         'C': 198,
         'D': 111,
         'E': 72,
         'F': 136,
         'G': 115,
         'H': 69,
         'I': 34,
         'J': 20,
         'K': 20,
         'L': 88,
         'M': 107,
         'N': 37,
         'O': 41,
         'P': 142,
         'Q': 23,
         'R': 105,
         'S': 366,
         'T': 149,
         'U': 33,
         'V': 43,
         'W': 83,
         'Y': 6,
         'Z': 3})

Initialise game

In [418]:
class Game:
    
    def __init__(self, df_all_5l_words):
        
        # Start with whole alphabet as list of possible letters in word
        self.possible_letters = list(string.ascii_uppercase)
        
        # To store guessed letters that are correct, but in the wrong location
        self.dict_misplaced_letters = Counter()
        
        # Answer space
        self.df_possible_5l_words = df_all_5l_words.copy(deep=True)
        
        # Dictionary of answers, initialised as empty
        self.dict_letters = defaultdict(str)
        for i in range(5):
            self.dict_letters[i+1] = None
        
        # Initialise dictionary of letter counts at each position, updated as we play the game
        self.dict_letter_counts = defaultdict(str)
        for i in range(5):
            self.dict_letter_counts[i+1] = Counter(df_all_5l_words[f'letter_{i+1}'])
        

    def calculate_freq_score(self, letters):
        '''
        Based on a set of letters, sum their frequency score at each position
        '''
        
        score = 0
        for i, l in enumerate(list(letters.upper())):
            score += self.dict_letter_counts[i+1][l]
            
        return score
        
    
    def guess(self):
        '''
        Given a game state, produces the best guess possible based on the 'score' at
        each position.
        
        The list of possible words is already filtered for correct answers,
        and letters that fall within the list of possible list of letters for the remaining
        positions.
        '''
        
        # First, reset letter counts
        for i in range(5):
            self.dict_letter_counts[i+1] = Counter(self.df_possible_5l_words[f'letter_{i+1}'])
        
        # Vectorize frequency score function
        vect_calculate_freq_score = np.vectorize(self.calculate_freq_score)
        
        self.df_possible_5l_words['freq_score'] = vect_calculate_freq_score(self.df_possible_5l_words['word'])
        
        self.df_possible_5l_words = self.df_possible_5l_words.sort_values(by='freq_score', ascending=False)
        return self.df_possible_5l_words
    
    
    def check_misplaced_letters(self, word):
        '''
        Allows us to filter dataframe of possible words containing at least the misplaced letters.
        
        e.g. If I guess 'GREET', and the 3rd  E is green, I filter the dataframe of possible words
        for position 3 == "E". But if the 4th E is yellow, I also want to filter the possible words for 
        any where E falls in the 1st, 2nd, or 5th spots.
        
        We would first filter for position 4 != "E" (outside of this function), then for a given word - 
        check that the count of the letter "E" was >=1 outside of position 3.
        
        Returns a boolean, where True implies that the word has AT LEAST the letters contained in 
        self.dict_misplaced_letters (e.g. 2 "E"s outside of position 3 in the above example would be acceptable).
        '''
        
        # Break into letters
        list_word = list(word)
        
        # Get indices (1 indexed) of positions that have not yet been solved
        not_solved = [key for key, value in self.dict_letters.items() if value is None]
        
        # Filter list of words for those not yet solved, removing 1 from the index as our letters are 1 indexed
        list_word_unsolved = [list_word[i-1] for i in not_solved]
        
        # Check counts
        dict_count_letters = Counter(list_word_unsolved)
        
        # Compare to dictionary of misplaced letters
        valid = True
        for check_key, check_value in self.dict_misplaced_letters.items():
            if dict_count_letters[check_key] < check_value:
                valid = False
                
        return valid
        
    
    def update(self, guess, results):
        '''
        Takes a 5 letter guess as a string, and a list of results in the format:
        0 - incorrect
        1 - right letter, wrong place
        2 - right letter, right place
        
        Updates the game states (possible letters, misplaced letters, answers).
        
        Doesn't return anything
        '''
        
        assert len(guess) == 5, 'Guess must be 5 characters long'
        
        # Convert guess into list of letters
        list_guess = list(guess.upper())
        
        # Zip with results
        df_guess_results = pd.DataFrame(data=list(zip(list_guess, results)),
                                        columns=['letter', 'result'],
                                        index=np.arange(1,6))
        
        # To prevent iterating through already solved letters
        already_solved = [key for key, value in self.dict_letters.items() if value is not None]
        

        # Update correct answers
        df_corr_answers = df_guess_results.query('result==2')
        if df_corr_answers.shape[0] > 0:
            for idx, row in df_corr_answers.iterrows():
                
                # Prevent updates for previously solved letters
                if idx in already_solved:
                    pass
                else:
                    corr_letter = row['letter']
                    self.dict_letters[idx] = corr_letter
                
                    # If correct letter was previously guessed as a misplaced letter, remove it
                    if corr_letter in self.dict_misplaced_letters.keys():
                        self.dict_misplaced_letters[corr_letter] -= 1
                        
                    # And filter dataframe of possible words
                    self.df_possible_5l_words = self.df_possible_5l_words.query(f'letter_{idx}=="{corr_letter}"')

          
        # Add misplaced letters to our list, if it's a new letter
        df_mispl_answers = df_guess_results.query('result==1')
        if df_mispl_answers.shape[0] > 0:
            
            # Filter dataframe to remove any words that have the misplaced letter in that column
            for idx, row in df_mispl_answers.iterrows():
                mispl_letter = row['letter']
                self.df_possible_5l_words = self.df_possible_5l_words.query(f'letter_{idx}!="{mispl_letter}"')  
            
            # Check how many we have of each letter that's misplaced
            guess_mispl_letters = df_mispl_answers['letter'].values
            dict_guess_mispl_letters = Counter(guess_mispl_letters)
            
            # Then update our dictionary of misplaced letters
            for key, value in dict_guess_mispl_letters.items():
                self.dict_misplaced_letters[key] = value   
            
            # Filter dataframe for words containing at least the count of the misplaced letters
            vect_check_misplaced_letters = np.vectorize(self.check_misplaced_letters)
            self.df_possible_5l_words['valid'] = vect_check_misplaced_letters(self.df_possible_5l_words['word'])
            self.df_possible_5l_words = self.df_possible_5l_words.query('valid == True')
            self.df_possible_5l_words = self.df_possible_5l_words.drop('valid', axis=1)    
        
        
        # Remove any incorrect letters from the list to guess from, if letter isn't in misplaced list
        df_wrong_answers = df_guess_results.query('result==0')
        if df_wrong_answers.shape[0] > 0:
            
            for l in df_wrong_answers['letter'].unique():
                if self.dict_misplaced_letters[l] == 0:
                    self.possible_letters.remove(l)
                
                
        # Finally, update list of possible 5 letter words by removing all rows where
        # for letters yet to be guessed, they don't fall in the list of possible letters
        yet_to_solve = [key for key, value in self.dict_letters.items() if value is None]
        for position in yet_to_solve:
            
            # Check all letters in a given position
            position_letters = self.df_possible_5l_words[f'letter_{position}']
            
            # Return a boolean list of whether that list is in the possible values or not
            position_in_possible_letters = [l in self.possible_letters for l in position_letters]
            
            # Filter
            self.df_possible_5l_words = self.df_possible_5l_words[position_in_possible_letters].copy(deep=True)
        
        # And recalibrate counter dictionary
        self.dict_most_likely = defaultdict(str)
        for i in range(5):
            self.dict_most_likely[i+1] = Counter(self.df_possible_5l_words[f'letter_{i+1}'])
        
MyGame = Game(df_words_5l)

Create a test function to play the game, given a target word to find

In [419]:
def play_game(target_word, debug=False):
    
    TestGame = Game(df_words_5l)
    
    target_letters = list(target_word.upper())
    
    for guess_turn in range(5):
        
        # Return word with highest frequency count across all letters as the guess
        guess_word = TestGame.guess().iloc[0]['word'] 
        guess_letters = list(guess_word)
        
        # Dictionary of results at each position
        dict_results = defaultdict(str)
        for i in range(5):
            dict_results[i] = None
        
        # First assign correct letters with a score of 2
        for pos, guess_letter in enumerate(guess_letters):
            if guess_letter == target_letters[pos]:
                dict_results[pos] = 2
        
        # For remaining letters, if they appear within the target word - count them as misplaced
        # Otherwise, count them as wrong
        remaining_pos = [key for key, value in dict_results.items() if value is None]
        if len(remaining_pos) == 0:
            # Guess correct
            results = [2, 2, 2, 2, 2]
        else:
            # Tag remaining letters in our guess and their position
            # Need to use list as we may have duplicate keys (same letter in >1 position)
            remaining_guess_letters = [[guess_letters[i], i] for i in remaining_pos]
            
            # Tag remaining letters in the answer, and create a Counter dictionary
            remaining_target_letters = [target_letters[i] for i in remaining_pos]
            dict_target_letter_count = Counter(remaining_target_letters)
            
            # Loop through our remaining guess letters
            for [letter, pos] in remaining_guess_letters:
                # Check they appear in the target word at least once
                if dict_target_letter_count[letter] > 0:
                    
                    # Subtract from target letter count, to prevent double counting
                    # e.g. tag first 'E' as misplaced, second 'E' as wrong when guessing GREET for
                    # target word STAGE
                    dict_target_letter_count[letter] -= 1
                    
                    # Update results
                    dict_results[pos] = 1
                else:
                    dict_results[pos] = 0
             
            # Turn into list 
            results = list(dict_results.values())
            
        # Finally, produce results as a list and pass back to game to update
        if debug:
            print(f'Turn {guess_turn+1}, guess {guess_word}, results {results}\n')
        if np.sum(results) == 10:
            if debug:
                print('Game won!')
            else:
                # Used for tracking metrics
                return (target_word, guess_turn+1)
            break
            
        # If the game isn't solved by turn 6, return "7" as the number of guesses
        if guess_turn == 5:
            if debug:
                print('Unsolved!')
            else:
                # Used for tracking metrics
                return (target_word, 7)
        
        TestGame.update(guess_word, results)

Test with a random sample of answers

In [422]:
play_game('BASIL', debug=True)

Turn 1, guess SLATE, results [1, 1, 1, 0, 0]

Turn 2, guess BASAL, results [2, 2, 2, 0, 2]

Turn 3, guess BASIL, results [2, 2, 2, 2, 2]

Game won!


In [421]:
rand_keys = np.random.randint(low=0, high=len(list_possible_answers), size=1)
rand_words = [list_possible_answers[k] for k in rand_keys]

for i, word in enumerate(rand_words):
    print(f'**GAME {i}: target {word}**')
    play_game(word, debug=True)
    print('\n')

**GAME 0: target PAGAN**
Turn 1, guess SLATE, results [0, 0, 1, 0, 0]

Turn 2, guess MARRY, results [0, 2, 0, 0, 0]

Turn 3, guess CANON, results [0, 2, 0, 0, 2]

Turn 4, guess PAGAN, results [2, 2, 2, 2, 2]

Game won!




Test across all answers, returning results

In [ ]:
all_results = []
for answer in tqdm(list_possible_answers):
    all_results.append(play_game(answer, debug=False))
all_results[:10]

 20%|██████████████████████████▍                                                                                                            | 453/2315 [00:26<02:16, 13.62it/s]

Convert to dataframe, and plot our results!

In [ ]:
df_all_results = pd.DataFrame(data=all_results,
                              columns=['word', 'turns_taken'])
df_all_results['turns_taken'].value_counts()

In [355]:
df_all_results.query('turns_taken==7')

,word,turns_taken
154,BASIC,7
155,BASIL,7
179,BELIE,7
183,BENCH,7
189,BEVEL,7
...,...,...
2233,WATER,7
2268,WIGHT,7
2276,WITCH,7
2283,WOOLY,7
